# T4 Clustering: algorisme K-mitjanes

# Índex

1. Clustering particional
2. Criteri suma d'errors quadràtics
3. Algorisme $K$-mitjanes de Duda i Hart
4. Algorisme $K$-mitjanes convencional

# 1 Clustering particional

**Clustering particional:** $\;$ donat un conjunt de $N$ dades $\mathcal{D}$ i un nombre de clústers $K,\,$ el clustering particional consisteix a optimitzar alguna funció criteri $J(\Pi)$ per a avaluar la qualitat de qualsevol partició $\Pi$ de les dades en $K$ clústers
$$\Pi^* = \operatorname*{argopt}_{\Pi} J(\Pi)$$

**Intractabilitat:** $\;$ el clustering particional és en general un problema intractable ja que el nombre de particions a explorar creix exponencialment amb $N$ i $K$ (veure [nombres d'Stirling del segon tipus](https://en.wikipedia.org/wiki/Stirling_numbers_of_the_second_kind))

**Aproximació usual:** $\;$ fem ús d'algorismes aproximats per a optimitzar un criteri particular com ara la suma d'errors quadràtics

# 2 Criteri suma d'errors quadràtics

**Suma d'errors quadràtics (SEQ):** $\;$ d'una partició $\Pi=\{X_1,\dotsc,X_K\}$
$$J(\Pi) = \sum_{k=1}^K J_k%
\quad\text{amb}\quad%
J_k = \sum_{\boldsymbol{x}\in X_k} \lVert{\boldsymbol{x}-\boldsymbol{m}_k}\rVert_2^2
\quad\text{i}\quad%
\boldsymbol{m}_k = \frac{1}{\lvert X_k\rvert}\sum_{\boldsymbol{x}\in X_k} \boldsymbol{x}$$

**Interpretació:**
* Cada clúster $k$ es representa pel seu **centroide** o **mitjana** $\,\boldsymbol{m}_k$
* Si $\boldsymbol{x}$ pertany al clúster $k$, $\boldsymbol{x}-\boldsymbol{m}_k$ és el **vector error** obtés al representar $\boldsymbol{x}$ amb $\boldsymbol{m}_k$
* L'error associat a $\boldsymbol{x}$ es mesura amb la norma Euclidiana del seu vector error, $\lVert{\boldsymbol{x}-\boldsymbol{m}_k}\rVert_2$
* Anomenem **distorsió** del clúster $k$, $\,J_k,\,$ a la suma d'errors al quadrat de les seues dades
* El criteri SEQ és la suma de les distorsions de tots els clústers i, òbviament, és un criteri a minimitzar
* Idealment, esperem clústers hiper-esfèrics compactes i de grandària pareguda, al voltant de $K$ mitjanes ben separades
* Si la partició natural de les dades és distinta a l'esperada, és probable que la minimització de la SEQ no la trobe

**Exemple:** $\;$ càlcul de la SEQ per a $\Pi=\{X_1=\{(1, 7)^t, (4, 2)^t, (4,6)^t\}, X_2=\{(8, 2)^t, (8, 6)^t\}\}$
$$\begin{align*}
\boldsymbol{m}_1&=(3,5)^t&&J_1=8+10+2=20\\
\boldsymbol{m}_2&=(8,4)^t&&J_2=4+4=8
\end{align*}$$
$$J=J_1+J_2=28$$

In [1]:
import numpy as np; np.set_printoptions(precision=2)
def SEQ(X, y): # labels from 0 to K-1 for simplicity
    N, D = X.shape; K = np.max(y)+1; J = 0.0; m = np.zeros((K, D)); S = np.zeros(K).astype(int)
    for k in range(K):
        Xk = np.squeeze(X[np.where(y==k),:]); S[k] = Xk.shape[0]; 
        m[k] = Xk.mean(axis=0); J += np.square(Xk - m[k]).sum()
    return J, m, S
X = np.array([[1, 7], [4, 2], [4, 6], [8, 2], [8, 6]]); y = np.array([0, 0, 0, 1, 1])
J, m, S = SEQ(X, y); print(f'J = {J:.2f} m = {m.reshape(1, -1)} S = {S}')

J = 28.00 m = [[3. 5. 8. 4.]] S = [3 2]


# 3 Algorisme $K$-mitjanes de Duda i Hart

**Increment de SEQ al transferir una dada de clúster:** $\;$ si es transfereix una dada $\boldsymbol{x}$ del clúster $i$ al $j$, l'increment de SEQ és
$$\Delta J = \frac{\lvert X_j\rvert}{\lvert X_j\rvert + 1}\lVert\boldsymbol{x}-\boldsymbol{m}_j\rVert_2^2-\frac{\lvert X_i\rvert}{\lvert X_i\rvert - 1}\lVert\boldsymbol{x}-\boldsymbol{m}_i\rVert_2^2$$

**Condició DH:** $\;$ convé transferir si $\Delta J<0,$ és a dir, si s'incrementa menys $J$ en $X_j$ del que es decrementa en $X_i$
$$\frac{\lvert X_j\rvert}{\lvert X_j\rvert + 1}\lVert\boldsymbol{x}-\boldsymbol{m}_j\rVert_2^2<\frac{\lvert X_i\rvert}{\lvert X_i\rvert - 1}\lVert\boldsymbol{x}-\boldsymbol{m}_i\rVert_2^2$$

**Algorisme $K$-mitjanes de Duda i Hart:** $\;$ per a cada dada, cerca la transferència de menor $\Delta J$ i l'aplica si compleix DH
> **Entrada:** $\;$ una partició inicial, $\,\Pi=\{X_1,\dotsc,X_K\}$ <br>
> **Eixida:** $\;$ una partició optimitzada, $\,\Pi^*=\{X_1,\dotsc,X_K\}$ <br>
> Calcular mitjanes i $J$ <br>
> `repetir`
>> `per a tota` $\;$ dada $\,\boldsymbol{x}$ <br>
>>> Siga $i$ el clúster en el qual es troba $\boldsymbol{x}$ <br>
>>> Trobar un $j\neq i$ que minimitze $\,\triangle J\,$ en transferir $\,\boldsymbol{x}\,$ d'$i$ a $j$ <br>
>>> `si` $\,\triangle J<0:\;$ transferir $\boldsymbol{x}$ d'$i$ a $j$ i actualitzar mitjanes i $J$ <br>
>
> `fins que` $\;$ no es trobe cap transferència profitosa

**Implementació:** $\;$ funció per a problemes senzills

In [2]:
import numpy as np; np.set_printoptions(precision=2, linewidth=np.inf)
def kmeansDH(X, y, max_iter=10, verbose=0):
    N = X.shape[0]; J, m, S = SEQ(X, y); z = y.copy(); notransfer = 0
    for iter in range(max_iter):
        for n in range(N):
            x = X[n, :]; i = z[n]
            if S[i] == 1: continue
            D = np.square(x - m).sum(axis=1); Di = S[i] / (S[i] - 1.0) * D[i]
            D = S / (S + 1.0) * D; D[i] = np.inf; j = np.argmin(D); Dj = D[j]; DJ = Dj - Di
            if verbose > 0: print(f'{iter} {x} {Di:.2f} {Dj:.2f} {DJ:.2f}', end=" ")
            if DJ < 0.0:
                z[n] = j; S[i] -= 1; S[j] += 1; J += DJ; notransfer = 1
                m[i] = m[i] - (x - m[i]) / S[i]; m[j] = m[j] + (x - m[j]) / S[j]
                if verbose > 0: print(f'=> z ={z} m = {m.reshape(1, -1)} J = {J:.2f}')
            else: print("=> no transfer"); notransfer += 1
            if notransfer == N: break
        if notransfer == N: break
    return J, m, z

**Exemple (cont.):** $\quad X_1=\{\boldsymbol{x}_1=(1, 7)^t, \boldsymbol{x}_2=(4, 2)^t, \boldsymbol{x}_3=(4,6)^t\}\quad X_2=\{\boldsymbol{x}_4=(8, 2)^t, \boldsymbol{x}_5=(8, 6)^t\}$
<center>

|$\boldsymbol{x}$|$i$|$j$|$\frac{\lvert X_i\rvert}{\lvert X_i\rvert - 1}\lVert\boldsymbol{x}-\boldsymbol{m}_i\rVert_2^2$|$\frac{\lvert X_j\rvert}{\lvert X_j\rvert + 1}\lVert\boldsymbol{x}-\boldsymbol{m}_j\rVert_2^2$|$\triangle J$|$X_1$|$X_2$|$\boldsymbol{m}_1$|$\boldsymbol{m}_2$|$J$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|||||||$\{\boldsymbol{x}_1,\boldsymbol{x}_2,\boldsymbol{x}_3\}$|$\{\boldsymbol{x}_4,\boldsymbol{x}_5\}$|$(3,5)^t$|$(8,4)^t$|$28$|
|$\boldsymbol{x}_1$|$1$|$2$|$\frac{3}{2}\cdot 8=12$|$\frac{2}{3}\cdot 58=38.67$|$\frac{80}{3}=26.67$||||
|$\boldsymbol{x}_2$|$1$|$2$|$\frac{3}{2}\cdot 10=15$|$\frac{2}{3}\cdot 20=13.33$|$-\frac{5}{3}=-1.67$|$\{\boldsymbol{x}_1,\boldsymbol{x}_3\}$|$\{\boldsymbol{x}_2,\boldsymbol{x}_4,\boldsymbol{x}_5\}$|$\left(\frac{5}{2},\frac{13}{2}\right)^t$|$\left(\frac{20}{3},\frac{10}{3}\right)^t$|$26.33$|
|$\boldsymbol{x}_3$|$1$|$2$|$\frac{2}{1}\cdot\frac{10}{4}=5$|$\frac{3}{4}\cdot\frac{128}{9}=10.67$|$\frac{17}{3}=5.67$|
|$\boldsymbol{x}_4$|$2$|$1$|$\frac{3}{2}\cdot\frac{32}{9}=5.33$|$\frac{2}{3}\cdot\frac{101}{2}=33.67$|$\frac{85}{3}=28.33$|
|$\boldsymbol{x}_5$|$2$|$1$|$\frac{3}{2}\cdot\frac{80}{9}=13.33$|$\frac{2}{3}\cdot\frac{61}{2}=20.33$|$7$|
|$\boldsymbol{x}_1$|$1$|$2$|$\frac{2}{1}\cdot\frac{5}{2}=5$|$\frac{3}{4}\cdot\frac{410}{9}=34.17$|$\frac{175}{6}=29.17$|

</center>

In [3]:
X = np.array([[1, 7], [4, 2], [4, 6], [8, 2], [8, 6]]); y = np.array([0, 0, 0, 1, 1])
J, m, z = kmeansDH(X, y, max_iter=3, verbose=1)
print(f'{z} {m.reshape(1, -1)} {J:.2f}')

0 [1 7] 12.00 38.67 26.67 => no transfer
0 [4 2] 15.00 13.33 -1.67 => z =[0 1 0 1 1] m = [[2.5  6.5  6.67 3.33]] J = 26.33
0 [4 6] 5.00 10.67 5.67 => no transfer
0 [8 2] 5.33 33.67 28.33 => no transfer
0 [8 6] 13.33 20.33 7.00 => no transfer
1 [1 7] 5.00 34.17 29.17 => no transfer
[0 1 0 1 1] [[2.5  6.5  6.67 3.33]] 26.33


# 4 Algorisme $K$-mitjanes convencional

**Condició convencional:** $\;$ convé transferir $\boldsymbol{x}$ del clúster $i$ al $j$ si
$$\lVert\boldsymbol{x}-\boldsymbol{m}_j\rVert_2^2<\lVert\boldsymbol{x}-\boldsymbol{m}_i\rVert_2^2$$

**Relació amb la condició DH:** $\;$ la convencional és suficient (però no necessària; veure exemple)
$$\frac{\lvert X_j\rvert}{\lvert X_j\rvert + 1}\lVert\boldsymbol{x}-\boldsymbol{m}_j\rVert_2^2<\lVert\boldsymbol{x}-\boldsymbol{m}_j\rVert_2^2\overset{?}{<}\lVert\boldsymbol{x}-\boldsymbol{m}_i\rVert_2^2<\frac{\lvert X_i\rvert}{\lvert X_i\rvert - 1}\lVert\boldsymbol{x}-\boldsymbol{m}_i\rVert_2^2$$

**Algorisme $K$-mitjanes convencional:**
> **Entrada:** $\;$ una partició inicial, $\,\Pi=\{X_1,\dotsc,X_K\}$ <br>
> **Eixida:** $\;$ una partició optimitzada, $\,\Pi^*=\{X_1,\dotsc,X_K\}$ <br>
> `repetir`
>> Calcular les mitjanes dels clústers <br>
>> Reclassificar les dades segons les mitjanes més properes <br>
>
> `fins que` $\;$ no es reclassifique cap dada

**Implementació:** $\;$ funció per a problemes senzills

In [4]:
import numpy as np; np.set_printoptions(precision=2, linewidth=np.inf)
def kmeans(X, y, max_iter=10, verbose=0):
    N = X.shape[0]; z_prev = y.copy(); z = y.copy()
    for iter in range(max_iter):
        J, m, _ = SEQ(X, z); np.copyto(z_prev, z); transfers = 0
        for n in range(N):
            x = X[n, :]; i = z[n]; D = np.square(x - m).sum(axis=1)
            Di = D[i]; D[i] = np.inf; j = np.argmin(D); Dj = D[j]
            if verbose > 0: print(f'{iter} {x} {Di:.2f} {Dj:.2f}', end=" ")
            if Dj < Di:
                z[n] = j; transfers += 1
                if verbose > 0: print(f'=> z ={z}')
            else: print("=> no transfer");
        if transfers == 0: break
    return J, m, z

**Exemple (cont.):** $\quad X_1=\{\boldsymbol{x}_1=(1, 7)^t, \boldsymbol{x}_2=(4, 2)^t, \boldsymbol{x}_3=(4,6)^t\}\quad X_2=\{\boldsymbol{x}_4=(8, 2)^t, \boldsymbol{x}_5=(8, 6)^t\}$
<center>

|$\boldsymbol{x}$|$i$|$j$|$\lVert\boldsymbol{x}-\boldsymbol{m}_i\rVert_2^2$|$\lVert\boldsymbol{x}-\boldsymbol{m}_j\rVert_2^2$|$X_1$|$X_2$|$\boldsymbol{m}_1$|$\boldsymbol{m}_2$|$J$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
||||||$\{\boldsymbol{x}_1,\boldsymbol{x}_2,\boldsymbol{x}_3\}$|$\{\boldsymbol{x}_4,\boldsymbol{x}_5\}$|$(3,5)^t$|$(8,4)^t$|$28$|
|$\boldsymbol{x}_1$|$1$|$2$|$8$|$58$||||
|$\boldsymbol{x}_2$|$1$|$2$|$10$|$20$|
|$\boldsymbol{x}_3$|$1$|$2$|$2$|$20$|
|$\boldsymbol{x}_4$|$2$|$1$|$34$|$4$|
|$\boldsymbol{x}_5$|$2$|$1$|$26$|$4$|

</center>

In [5]:
X = np.array([[1, 7], [4, 2], [4, 6], [8, 2], [8, 6]]); y = np.array([0, 0, 0, 1, 1])
J, m, z = kmeans(X, y, max_iter=1, verbose=1)
print(f'{z} {m.reshape(1, -1)} {J:.2f}')

0 [1 7] 8.00 58.00 => no transfer
0 [4 2] 10.00 20.00 => no transfer
0 [4 6] 2.00 20.00 => no transfer
0 [8 2] 4.00 34.00 => no transfer
0 [8 6] 4.00 26.00 => no transfer
[0 0 0 1 1] [[3. 5. 8. 4.]] 28.00
